In [1]:
from telegram import (ReplyKeyboardMarkup, ReplyKeyboardRemove)
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters, RegexHandler,
                          ConversationHandler)
import requests
import logging
import pandas as pd
import numpy as np

In [2]:
mapping_target = {
    'A': 0,
    'A+': 1,
    'B': 2,
    'C': 3
}

ddd = {}
for k, v in mapping_target.items():
    ddd[float(v)] = k
    
df = pd.read_csv('v2.csv')
df['object_class_fixed'] = df.object_class_fixed.apply(lambda x:ddd[x])

In [3]:
# df[df['latitude'] == np.inf]

In [4]:
def _parse(d, path, i=0):

    if isinstance(path, str):
        path = path.split(':')

    if i == len(path):
        return d

    for j in range(len(path)):
        if path[i + j].isdigit():
            path[i + j] = int(path[i + j])

        d = d[path[i + j]]
        return _parse(d, path, i=i+1)

def parse(d, path):

    try:
        return _parse(d, path)
    except Exception as e:
        print('some error during path travesing: {}'.format(e))
        return None

In [5]:


from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters, RegexHandler,
                          ConversationHandler)

import logging

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

CHOOSING, TYPING_REPLY, TYPING_CHOICE = range(3)

reply_keyboard = [['Адрес', 'FAQ'],
                  ['Done']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)


def facts_to_str(user_data):
    facts = list()

    for key, value in user_data.items():
        facts.append('{} - {}'.format(key, value))

    return "\n".join(facts).join(['\n', '\n'])


def start(bot, update):
    update.message.reply_text(
        "Найду инфу по классу недвижки и еще много стафа",
        reply_markup=markup)

    return CHOOSING


def regular_choice(bot, update, user_data):
    text = update.message.text
    user_data['choice'] = text
    update.message.reply_text(
        'Ваш {}? Нужно понять, что мне делать!'.format(text.lower()))

    return TYPING_REPLY


# def custom_choice(bot, update):
#     update.message.reply_text('Alright, please send me the category first, '
#                               'for example "Most impressive skill"')

#     return TYPING_CHOICE

from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))


# * longitude 'геокоординаты долгота'
# * latitude 'геокоординаты широта'

def closest(data, v):
    
    minim = np.inf
    min_row = None
    for i, row in data.iterrows():
        dist = distance(v['latitude'], v['longitude'], row['latitude'], row['longitude'])
        if dist < minim:
            minim = dist
            min_row = row
    return min_row
            
# tempDataList = [{'lat': 39.7612992, 'lon': -86.1519681}, 
#                 {'lat': 39.762241,  'lon': -86.158436 }, 
#                 {'lat': 39.7622292, 'lon': -86.1578917}]

# v = {'lat': 39.7622290, 'lon': -86.1519750}
# print(closest(tempDataList, v))


def received_information(bot, update, user_data):
    text = update.message.text
    category = user_data['choice']
    user_data[category] = text
    del user_data['choice']
    
    r = requests.get('https://geocode-maps.yandex.ru/1.x/?geocode={}&format=json'.format(text)).json()
    
    found = parse(r, path='response:GeoObjectCollection:metaDataProperty:GeocoderResponseMetaData:found')
    if found == 0:
        

        update.message.reply_text("Угараете? Не представляю где такое ищут", reply_markup=markup)
        
    else:

        update.message.reply_text("Что - то дайте подумать на этим:"
                                  "{}"
                                  "Нужно глянуть в бдшечку.".format(facts_to_str(user_data)), reply_markup=markup)
        
        data_pos = parse(r, path='response:GeoObjectCollection:featureMember:0:GeoObject:Point:pos')
        if data_pos:
#             try:
            lat, lon = map(np.float, data_pos.split(' '))
            update.message.reply_text("вот эта точка"
                                      " {} "
                                      "".format(data_pos), reply_markup=markup)
                
            v = {'latitude': lat, 'longitude': lon}
            print(v)
            update.message.reply_text("вот это вот"
                                      " {} "
                                      "".format(closest(df, v)), reply_markup=markup)

                
                
#             except:
#                 update.message.reply_text("Сложно что-то координаты определить", reply_markup=markup)
                
    

    return CHOOSING


def done(bot, update, user_data):
    if 'choice' in user_data:
        del user_data['choice']

    update.message.reply_text("Понял:"
                              "{}"
                              "До скорого!".format(facts_to_str(user_data)))

    user_data.clear()
    return ConversationHandler.END


def error(bot, update, error):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, error)


def main():
    # Create the Updater and pass it your bot's token.
    updater = Updater("503183548:AAEx9UHfJXbdXl8kIhPxOJAjr0ejk8RIp5c")

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states GENDER, PHOTO, LOCATION and BIO
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            CHOOSING: [RegexHandler('^(Адрес|FAQ)$',
                                    regular_choice,
                                    pass_user_data=True),
                       ],

            TYPING_CHOICE: [MessageHandler(Filters.text,
                                           regular_choice,
                                           pass_user_data=True),
                            ],

            TYPING_REPLY: [MessageHandler(Filters.text,
                                          received_information,
                                          pass_user_data=True),
                           ],
        },

        fallbacks=[RegexHandler('^Ясненько$', done, pass_user_data=True)]
    )

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


In [ ]:
main()

2018-02-18 16:35:37,319 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
2018-02-18 16:35:37,320 - __main__ - WARNING - Update "None" caused error "Timed out"
2018-02-18 16:43:06,914 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
2018-02-18 16:43:06,916 - __main__ - WARNING - Update "None" caused error "Timed out"
2018-02-18 16:43:20,261 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
2018-02-18 16:43:20,262 - __main__ - WARNING - Update "None" caused error "Timed out"
2018-02-18 16:43:34,027 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
2018-02-18 16:43:34,028 - __main__ - WARNING - Update "None" caused error "Timed out"
2018-02-18 16:44:10,400 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
2018-02-18 16:44:10,401 - __main__ - WARNING - Update "None" caused error "Timed out"
2018-02-18 16:44:23,680 - telegram.ext.updater - ERROR - Error while getting Updates: Time

2018-02-18 16:50:57,297 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError EOF occurred in violation of protocol (_ssl.c:748)"
2018-02-18 16:52:57,408 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError EOF occurred in violation of protocol (_ssl.c:748)
2018-02-18 16:52:57,410 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError EOF occurred in violation of protocol (_ssl.c:748)"
2018-02-18 17:50:48,106 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
2018-02-18 17:50:48,109 - __main__ - WARNING - Update "None" caused error "Timed out"
2018-02-18 17:51:30,288 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
2018-02-18 17:51:30,290 - __main__ - WARNING - Update "None" caused error "Timed out"
2018-02-18 18:00:10,384 - telegram.ext.updater - ERROR - Error while getting Updates: Timed out
2018-02-18 18:00:10,385 - __main__ - WARNING - Update "None" caused error "Timed out"
2018-02-1